In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler

In [29]:
df = pd.read_csv('dataset_feature_engineering.csv')

#### Data Transformations

In [30]:
# We transform every string value to lowercase
df['merchant'] = df['merchant'].str.lower()
df['category'] = df['category'].str.lower()
df['first'] = df['first'].str.lower()
df['last'] = df['last'].str.lower()
df['gender'] = df['gender'].str.lower()
df['state'] = df['state'].str.lower()
df['street'] = df['street'].str.lower()
df['city'] = df['city'].str.lower()
df['job'] = df['job'].str.lower()

In [ ]:
# We transform every date value to datetime
df['dob'] = pd.to_datetime(df['dob'], format='%Y-%m-%d')

# We transform the 'unix_time' column to datetime
df['unix_time'] = pd.to_datetime(df['unix_time'], unit='s')

In [ ]:
# Export the dataset to a CSV file
# df.to_csv('dataset_cleaned.csv', index=False)

In [33]:
df

,cc_num,merchant,category,amt,first,last,gender,street,city,state,...,first_time_at_merchant,dist_between_client_and_merch,trans_month,trans_day,hour,year,times_shopped_at_merchant,times_shopped_at_merchant_year,times_shopped_at_merchant_month,times_shopped_at_merchant_day
0,2703186189652095,"fraud_rippin, kub and mann",misc_net,4.97,jennifer,banks,f,561 perry cove,moravian falls,nc,...,True,78.773821,1,1,0,2019,5,4,2,1
1,630423337322,"fraud_heller, gutmann and zieme",grocery_pos,107.23,stephanie,gill,f,43039 riley greens suite 393,orient,wa,...,True,30.216618,1,1,0,2019,4,4,1,1
2,38859492057661,fraud_lind-buckridge,entertainment,220.11,edward,sanchez,m,594 white dale suite 530,malad city,id,...,True,108.102912,1,1,0,2019,4,3,1,1
3,3534093764340240,"fraud_kutch, hermiston and farrell",gas_transport,45.00,jeremy,white,m,9443 cynthia court apt. 038,boulder,mt,...,True,95.685115,1,1,0,2019,1,1,1,1
4,375534208663984,fraud_keeling-crist,misc_pos,41.96,tyler,garcia,m,408 bradley rest,doe hill,va,...,True,77.702395,1,1,0,2019,6,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1852389,30560609640617,fraud_reilly and sons,health_fitness,43.77,michael,olson,m,558 michael estates,luray,mo,...,False,77.032467,12,3,23,2020,6,3,1,1
1852390,3556613125071656,fraud_hoppe-parisian,kids_pets,111.84,jose,vasquez,m,572 davis mountains,lake jackson,tx,...,False,100.023736,12,3,23,2020,5,3,1,1
1852391,6011724471098086,fraud_rau-robel,kids_pets,86.88,ann,lawson,f,144 evans islands apt. 683,burbank,wa,...,False,80.887812,12,3,23,2020,10,7,1,2
1852392,4079773899158,fraud_breitenberg llc,travel,7.99,eric,preston,m,7020 doyle stream apt. 951,mesa,id,...,False,53.060882,12,3,23,2020,4,2,2,1


In [34]:
df.shape

(1852394, 35)

### Feature Engineering

In [ ]:
# We sort the DataFrame by the card number and time
df = df.sort_values(['cc_num', 'unix_time'])
# We calculate the time difference in seconds between consecutive transactions per card
df['time_diff_seconds'] = df.groupby('cc_num')['unix_time'].diff().dt.total_seconds().fillna(0)
# This to know how often the client spends between transactions

# We calculate the hour window by unix time
df['hour_window'] = df['unix_time'].dt.floor('h')
# We calculate the amount of transactions per hour made per card
df['trans_per_hour'] = df.groupby(['cc_num', 'hour_window'])['trans_num'].transform('count')
# This to know how many transactions are made per hour by the client

# Ratio of transactions per hour to total transactions per card
df['hour_trans_ratio'] = df.groupby(['cc_num', 'hour'])['trans_num'].transform('count') / df.groupby('cc_num')['trans_num'].transform('count')
# This to know how many transactions the client usually makes per hour

In [ ]:
# We calculate if the distance between the client and the merchant is unusual (> 100 km)
df['unusual_distance'] = (df['dist_between_client_and_merch'] > 100).astype(int)
# This to know if the far transactions made by the client

# We calculate the distance between the client and the merchant
df['distance_diff'] = df.groupby('cc_num')['dist_between_client_and_merch'].diff().fillna(0)
# We calculate the velocity in km/h between consecutive transactions
df['velocity_km_h'] = (df['distance_diff'] / (df['time_diff_seconds'] / 3600)).replace([float('inf'), -float('inf')], 0).fillna(0)
# This to know how fast the client most move to make those transactions

# We calculate the mean and standard deviation of the distance between the client and the merchant
df['mean_dist_merchant'] = df.groupby('merchant')['dist_between_client_and_merch'].transform('mean')
df['std_dist_merchant'] = df.groupby('merchant')['dist_between_client_and_merch'].transform('std')
df['dist_z_score'] = (df['dist_between_client_and_merch'] - df['mean_dist_merchant']) / df['std_dist_merchant'].replace(0, 1)
# This to know how unusual the distance between the client and the merchant is

In [ ]:
# We calculate the amount ratio of transactions per month and year
df['amt_month_ratio'] = df['amt'] / df['amt_month'].replace(0, 1)
df['amt_year_ratio'] = df['amt'] / df['amt_year'].replace(0, 1)
# This helps us identify amounts that are significantly higher or lower than the average for that month or year.

# Desviación de monto por categoría
df['mean_amt_category'] = df.groupby('category')['amt'].transform('mean')
df['std_amt_category'] = df.groupby('category')['amt'].transform('std')
df['amt_z_score'] = (df['amt'] - df['mean_amt_category']) / df['std_amt_category'].replace(0, 1)
# This helps us identify transactions that are significantly higher or lower than the average for that category, which usually is the behavior of fraudulent transactions.

# Primera transacción de alto monto
amt_threshold = df['amt'].quantile(0.9)
df['high_amt_first_time'] = (df['first_time_at_merchant'] & (df['amt'] > amt_threshold)).astype(int)
# This to know if the client made a high amount transaction for the first time at the merchant, which could mean a fraudulent transaction.

In [ ]:
# We calculate the mean, standard deviation, and z-score of the amount of times spent at each merchant
df['mean_times_day_merchant'] = df.groupby('merchant')['times_shopped_at_merchant_day'].transform('mean')
df['std_times_day_merchant'] = df.groupby('merchant')['times_shopped_at_merchant_day'].transform('std')
df['times_day_z_score'] = (df['times_shopped_at_merchant_day'] - df['mean_times_day_merchant']) / df['std_times_day_merchant'].replace(0, 1)
# This helps us identify transactions that were made at the merchant that ocurred more frequently than usual, which usually is the behavior of fraudulent transactions.

# Proporción de fraudes por comerciante (en datos de entrenamiento)
df['fraud_rate_merchant'] = df.groupby('merchant')['is_fraud'].transform('mean')
# This helps us identify merchants that have a high rate of fraud, which could indicate that the merchant is where the cards information is being stolen.

In [ ]:
# We calculate the amount of unique cards that were used at each merchant
df['unique_cards_per_hour'] = df.groupby(['merchant', 'hour_window'])['cc_num'].transform('nunique')
# This helps us identify if there is an usually high number of unique cards that were used at the merchant, which could indicate that a coordinated attack took place during a specific time.


# We calculate the variance of the amount spent in each hour window
df['amt_variance_hour'] = df.groupby(['merchant', 'hour_window'])['amt'].transform('std').fillna(0)
# This could helps us identify if there is a coordinated attack taking place by looking at the variance of the amount spent in each hour window. A very high amount could indicate that there are suspiciously high transactions being made and if the value is too low, it could indicate that there are many stolen cards that are being tested (which is used to check if a stolen card can actually make any purchase).

In [40]:
df

,cc_num,merchant,category,amt,first,last,gender,street,city,state,...,mean_amt_category,std_amt_category,amt_z_score,high_amt_first_time,mean_times_day_merchant,std_times_day_merchant,times_day_z_score,fraud_rate_merchant,unique_cards_per_hour,amt_variance_hour
1017,60416207185,"fraud_jones, sawayn and romaguera",misc_net,7.27,mary,diaz,f,9886 anita drive,fort washakie,wy,...,80.181370,165.896953,-0.439498,0,1.411340,0.660168,-0.623083,0.007782,1,0.000000
2724,60416207185,fraud_berge llc,gas_transport,52.94,mary,diaz,f,9886 anita drive,fort washakie,wy,...,63.477271,15.869363,-0.664001,0,1.967776,1.140589,0.028252,0.002905,1,0.000000
2726,60416207185,fraud_luettgen plc,gas_transport,82.08,mary,diaz,f,9886 anita drive,fort washakie,wy,...,63.477271,15.869363,1.172242,0,1.929820,1.082737,0.988403,0.004302,1,0.000000
2882,60416207185,fraud_daugherty llc,kids_pets,34.79,mary,diaz,f,9886 anita drive,fort washakie,wy,...,57.527851,48.729840,-0.466610,0,1.652921,0.852825,-0.765598,0.002499,2,5.579073
2907,60416207185,fraud_beier and sons,home,27.18,mary,diaz,f,9886 anita drive,fort washakie,wy,...,58.188180,48.504225,-0.639288,0,1.724513,0.873720,0.315303,0.001145,1,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1849259,4992346398065154184,fraud_parisian and sons,gas_transport,66.11,benjamin,kim,m,920 patrick light,mc nabb,il,...,63.477271,15.869363,0.165900,0,1.971607,1.162635,-0.835694,0.006773,1,0.000000
1849567,4992346398065154184,fraud_bogisich-homenick,misc_net,4.58,benjamin,kim,m,920 patrick light,mc nabb,il,...,80.181370,165.896953,-0.455713,0,1.445783,0.697994,-0.638663,0.014786,1,0.000000
1850234,4992346398065154184,fraud_erdman-kertzmann,gas_transport,95.96,benjamin,kim,m,920 patrick light,mc nabb,il,...,63.477271,15.869363,2.046883,0,1.968481,1.128436,0.914113,0.004168,2,28.220632
1850235,4992346398065154184,fraud_stracke-lemke,grocery_pos,149.48,benjamin,kim,m,920 patrick light,mc nabb,il,...,116.640146,52.726259,0.622837,0,1.751850,0.938109,-0.801452,0.013944,1,0.000000


In [41]:
df.shape

(1852394, 57)